## DUSP1 smiFISH Classification Workflow

This notebook processes experimental data from BigFish and CellProperties CSV files to classify DUSP1 smiFISH spots. Below is an outline of the workflow:

### Input:
- BigFish CSV files
- CellProperties CSV files

### Workflow Steps:
1. **Merge Experimental Data**  
    Combine data from the input CSV files for unified analysis.

2. **Signal-to-Noise Ratio (SNR) Analysis**  
    Perform SNR analysis to classify and filter spots based on signal quality.

3. **Measurement Analysis**  
    Conduct measurement analysis to extract relevant features from the data.

4. **Data Merging**  
    Merge all processed data into a single dataset for further analysis.

5. **Create Training Spot Crops (11px x 11px)**  
    Generate training data for machine learning models:
    - Select 1000 spots from each `h5_idx`, ensuring variation across cells and fields of view (FOVs).
    - Use the DUSP1 Display Manager to safely load spot channel images.
    - Perform max projection of images along the z-axis to create 2D (x, y) representations.
    - Extract 11px x 11px crops centered on each spot.
    - Rescale intensity for visualization purposes.
    - Display 100 sample spot crops for quality inspection before saving all crops to a directory.

In [ ]:
import h5py
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import dask.array as da
import os
import sys
import logging
import seaborn as sns
import datetime

# Today's date
today = datetime.date.today()
# Format date as 'Mar21' (for example)
date_str = today.strftime("%b%d")

logging.getLogger('matplotlib.font_manager').disabled = True
numba_logger = logging.getLogger('numba')
numba_logger.setLevel(logging.WARNING)

matplotlib_logger = logging.getLogger('matplotlib')
matplotlib_logger.setLevel(logging.WARNING)

src_path = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
print(src_path)
sys.path.append(src_path)

from src.Analysis_DUSP1_v2 import DUSP1AnalysisManager, SNRAnalysis, DUSP1Measurement, DUSP1DisplayManager, SpotCropSampler, DUSP1_filtering

In [ ]:
loc = None
log_location = r'/Volumes/share/Users/Eric/GR_DUSP1_reruns'
save_dir = r'/Volumes/share/Users/Eric/DUSP1_SpotCropsV2'
df_outut = r'/Volumes/share/Users/Eric/DUSP1_SpotCropsV2/SNR_dataframes'
if not os.path.exists(df_outut):
    os.makedirs(df_outut)
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

# Define Thresholds
abs_threshold = 4
mg_threshold = 3

# crop size
pad = 2

    # DUSP1 Replica D 3hr 100nM time-sweep R1

In [ ]:
am = DUSP1AnalysisManager(location=loc, log_location=log_location, mac=True) 
am.select_analysis('DUSP1_D_Final')

spots_df = am.select_datasets("spotresults", dtype="dataframe")
clusters_df = am.select_datasets("clusterresults", dtype="dataframe")
props_df = am.select_datasets("cell_properties", dtype="dataframe")

snr_df = SNRAnalysis(spots_df, props_df, clusters_df, abs_threshold=abs_threshold, mg_threshold=mg_threshold)

merged_spots_df, merged_clusters_df, merged_cellprops_df = snr_df.get_results()

# Create an instance of the DUSP1Measurement class.
dusp = DUSP1Measurement(merged_spots_df, merged_clusters_df, merged_cellprops_df)

# Process the data with a chosen threshold method
cell_level_results = dusp.measure(abs_threshold=abs_threshold, mg_threshold=mg_threshold)

# Add replica level unique IDs for 'unique_cell_id', 'unique_spot_id', and 'unique_cluster_id'
# Get number of digits in the max unique_cell_id
max_id = merged_cellprops_df['unique_cell_id'].max()
num_digits = len(str(max_id))

# Calculate multiplier to add a '1' followed by the right number of zeroes - prefix is specific for each experiment (e.g., repD:1, repE:2, etc.)
rep_prefix = 10
prefix = rep_prefix ** num_digits  # e.g., if max_id = 30245 → prefix = 100000

# Apply prefix to all related DataFrames
merged_spots_df['unique_cell_id'] += prefix
merged_clusters_df['unique_cell_id'] += prefix
merged_cellprops_df['unique_cell_id'] += prefix
cell_level_results['unique_cell_id'] += prefix

# Repeat for unique_spot_id and unique_cluster_id
max_spot_id = merged_spots_df['unique_spot_id'].max()
spot_prefix = rep_prefix ** len(str(max_spot_id))
merged_spots_df['unique_spot_id'] += spot_prefix

max_cluster_id = merged_clusters_df['unique_cluster_id'].max()
cluster_prefix = rep_prefix ** len(str(max_cluster_id))
merged_clusters_df['unique_cluster_id'] += cluster_prefix

# Save all results to CSV
rep_string = 'DUSP1_D_Final'
output_dir = df_outut
os.makedirs(output_dir, exist_ok=True)
cell_level_results.to_csv(os.path.join(output_dir, f"{rep_string}_cell_level_results_MG{mg_threshold}_Abs{abs_threshold}_{date_str}.csv"), index=False)
merged_spots_df.to_csv(os.path.join(output_dir, f"{rep_string}_merged_spots_df_MG{mg_threshold}_Abs{abs_threshold}_{date_str}.csv"), index=False)
merged_clusters_df.to_csv(os.path.join(output_dir, f"{rep_string}_merged_clusters_df_MG{mg_threshold}_Abs{abs_threshold}_{date_str}.csv"), index=False)
merged_cellprops_df.to_csv(os.path.join(output_dir, f"{rep_string}_merged_cellprops_df_MG{mg_threshold}_Abs{abs_threshold}_{date_str}.csv"), index=False)

# instantiate sampler
sampler = SpotCropSampler(
    spots_df=merged_spots_df,
    clusters_df=merged_clusters_df,
    cellprops_df=merged_cellprops_df,
    mount_prefix="/Volumes/share"
)

# # — TEST RUN: only 3 displays, no files written —
# crops, meta = sampler.run(
#     save_dir=save_dir,
#     display=1,
#     save_individual=False,
#     save_summary=False,
#     pad=2,
#     cells_per_quad=1,
#     spots_per_cell=20,
#     spotChannel=0
# )

# choose a prefix for this experiment
prefix = "DUSP1_D_Final_MG3_Abs4"

# full run, no per-spot files, but summary files with prefix
crops, meta = sampler.run(
    save_dir=save_dir,
    display=0,
    save_individual=False,
    save_summary=True,
    file_prefix=prefix,
    pad=pad,
    cells_per_quad=1,
    spots_per_cell=20,
    spotChannel=0
)

In [ ]:
# # Directory where you saved the summary files
# save_dir = "/Volumes/share/Users/Eric/DUSP1_SpotCrops"

# # 1) Load the data
# crops = np.load(os.path.join(save_dir, f"{prefix}_all_crops.npy"))  # shape: (N, 7, 7)
# meta  = pd.read_csv(os.path.join(save_dir, f"{prefix}_all_crop_metadata.csv"))

# # 2) Choose a few random examples (up to 5)
# num_examples = min(5, len(crops))
# indices = np.random.choice(len(crops), size=num_examples, replace=False)

# # 3) Display them
# fig, axes = plt.subplots(1, num_examples, figsize=(num_examples * 3, 3))
# for ax, idx in zip(axes, indices):
#     patch = crops[idx]
#     info  = meta.iloc[idx]
    
#     ax.imshow(patch, cmap='gray')
#     ax.set_title(
#         f"Cell {info.unique_cell_id}\n"
#         f"Spot {info.unique_spot_id}\n"
#         f"MG_SNR={info.MG_SNR:.1f}, SNR={info.snr:.1f}\n"
#         f"MG_pass={info.MG_pass}"
#     )
#     ax.axis('off')

# plt.tight_layout()
# plt.show()

    # DUSP1 Replica E 3hr 100nM time-sweep R2

In [ ]:
am = DUSP1AnalysisManager(location=loc, log_location=log_location, mac=True) 
am.select_analysis('DUSP1_E_Final')

spots_df = am.select_datasets("spotresults", dtype="dataframe")
clusters_df = am.select_datasets("clusterresults", dtype="dataframe")
props_df = am.select_datasets("cell_properties", dtype="dataframe")

snr_df = SNRAnalysis(spots_df, props_df, clusters_df, abs_threshold=abs_threshold, mg_threshold=mg_threshold)

merged_spots_df, merged_clusters_df, merged_cellprops_df = snr_df.get_results()

# Create an instance of the DUSP1Measurement class.
dusp = DUSP1Measurement(merged_spots_df, merged_clusters_df, merged_cellprops_df)

# Process the data with a chosen threshold method
cell_level_results = dusp.measure(abs_threshold=abs_threshold, mg_threshold=mg_threshold)

# Add replica level unique IDs for 'unique_cell_id', 'unique_spot_id', and 'unique_cluster_id'
# Get number of digits in the max unique_cell_id
max_id = merged_cellprops_df['unique_cell_id'].max()
num_digits = len(str(max_id))

# Calculate multiplier to add a '1' followed by the right number of zeroes - prefix is specific for each experiment (e.g., repD:1, repE:2, etc.)
rep_prefix = 20
prefix = rep_prefix ** num_digits  # e.g., if max_id = 30245 → prefix = 100000

# Apply prefix to all related DataFrames
merged_spots_df['unique_cell_id'] += prefix
merged_clusters_df['unique_cell_id'] += prefix
merged_cellprops_df['unique_cell_id'] += prefix
cell_level_results['unique_cell_id'] += prefix

# Repeat for unique_spot_id and unique_cluster_id
max_spot_id = merged_spots_df['unique_spot_id'].max()
spot_prefix = rep_prefix ** len(str(max_spot_id))
merged_spots_df['unique_spot_id'] += spot_prefix

max_cluster_id = merged_clusters_df['unique_cluster_id'].max()
cluster_prefix = rep_prefix ** len(str(max_cluster_id))
merged_clusters_df['unique_cluster_id'] += cluster_prefix

# Save all results to CSV
rep_string = 'DUSP1_E_Final'
output_dir = df_outut
os.makedirs(output_dir, exist_ok=True)
cell_level_results.to_csv(os.path.join(output_dir, f"{rep_string}_cell_level_results_MG{mg_threshold}_Abs{abs_threshold}_{date_str}.csv"), index=False)
merged_spots_df.to_csv(os.path.join(output_dir, f"{rep_string}_merged_spots_df_MG{mg_threshold}_Abs{abs_threshold}_{date_str}.csv"), index=False)
merged_clusters_df.to_csv(os.path.join(output_dir, f"{rep_string}_merged_clusters_df_MG{mg_threshold}_Abs{abs_threshold}_{date_str}.csv"), index=False)
merged_cellprops_df.to_csv(os.path.join(output_dir, f"{rep_string}_merged_cellprops_df_MG{mg_threshold}_Abs{abs_threshold}_{date_str}.csv"), index=False)

# instantiate sampler
sampler = SpotCropSampler(
    spots_df=merged_spots_df,
    clusters_df=merged_clusters_df,
    cellprops_df=merged_cellprops_df,
    mount_prefix="/Volumes/share"
)

# # — TEST RUN: only 3 displays, no files written —
# crops, meta = sampler.run(
#     save_dir=save_dir,
#     display=3,
#     save_individual=False,
#     save_summary=False,
#     pad=pad,
#     cells_per_quad=1,
#     spots_per_cell=20,
#     spotChannel=0
# )

# choose a prefix for this experiment
prefix = "DUSP1_E_Final_MG3_Abs4"

# full run, no per-spot files, but summary files with prefix
crops, meta = sampler.run(
    save_dir=save_dir,
    display=0,
    save_individual=False,
    save_summary=True,
    file_prefix=prefix,
    pad=pad,
    cells_per_quad=1,
    spots_per_cell=20,
    spotChannel=0
)

    # DUSP1 Replica F 3hr 100nM time-sweep R3

In [ ]:
am = DUSP1AnalysisManager(location=loc, log_location=log_location, mac=True) 
am.select_analysis('DUSP1_F_Final')

spots_df = am.select_datasets("spotresults", dtype="dataframe")
clusters_df = am.select_datasets("clusterresults", dtype="dataframe")
props_df = am.select_datasets("cell_properties", dtype="dataframe")

snr_df = SNRAnalysis(spots_df, props_df, clusters_df, abs_threshold=abs_threshold, mg_threshold=mg_threshold)

merged_spots_df, merged_clusters_df, merged_cellprops_df = snr_df.get_results()

# Create an instance of the DUSP1Measurement class.
dusp = DUSP1Measurement(merged_spots_df, merged_clusters_df, merged_cellprops_df)

# Process the data with a chosen threshold method
cell_level_results = dusp.measure(abs_threshold=abs_threshold, mg_threshold=mg_threshold)

# Add replica level unique IDs for 'unique_cell_id', 'unique_spot_id', and 'unique_cluster_id'
# Get number of digits in the max unique_cell_id
max_id = merged_cellprops_df['unique_cell_id'].max()
num_digits = len(str(max_id))

# Calculate multiplier to add a '1' followed by the right number of zeroes - prefix is specific for each experiment (e.g., repD:1, repE:2, etc.)
rep_prefix = 30
prefix = rep_prefix ** num_digits  # e.g., if max_id = 30245 → prefix = 100000

# Apply prefix to all related DataFrames
merged_spots_df['unique_cell_id'] += prefix
merged_clusters_df['unique_cell_id'] += prefix
merged_cellprops_df['unique_cell_id'] += prefix
cell_level_results['unique_cell_id'] += prefix

# Repeat for unique_spot_id and unique_cluster_id
max_spot_id = merged_spots_df['unique_spot_id'].max()
spot_prefix = rep_prefix ** len(str(max_spot_id))
merged_spots_df['unique_spot_id'] += spot_prefix

max_cluster_id = merged_clusters_df['unique_cluster_id'].max()
cluster_prefix = rep_prefix ** len(str(max_cluster_id))
merged_clusters_df['unique_cluster_id'] += cluster_prefix

# Save all results to CSV
rep_string = 'DUSP1_F_Final'
output_dir = df_outut
os.makedirs(output_dir, exist_ok=True)
cell_level_results.to_csv(os.path.join(output_dir, f"{rep_string}_cell_level_results_MG{mg_threshold}_Abs{abs_threshold}_{date_str}.csv"), index=False)
merged_spots_df.to_csv(os.path.join(output_dir, f"{rep_string}_merged_spots_df_MG{mg_threshold}_Abs{abs_threshold}_{date_str}.csv"), index=False)
merged_clusters_df.to_csv(os.path.join(output_dir, f"{rep_string}_merged_clusters_df_MG{mg_threshold}_Abs{abs_threshold}_{date_str}.csv"), index=False)
merged_cellprops_df.to_csv(os.path.join(output_dir, f"{rep_string}_merged_cellprops_df_MG{mg_threshold}_Abs{abs_threshold}_{date_str}.csv"), index=False)

# instantiate sampler
sampler = SpotCropSampler(
    spots_df=merged_spots_df,
    clusters_df=merged_clusters_df,
    cellprops_df=merged_cellprops_df,
    mount_prefix="/Volumes/share"
)

# # — TEST RUN: only 3 displays, no files written —
# crops, meta = sampler.run(
#     save_dir=save_dir,
#     display=3,
#     save_individual=False,
#     save_summary=False,
#     pad=pad,
#     cells_per_quad=1,
#     spots_per_cell=20,
#     spotChannel=0
# )

# choose a prefix for this experiment
prefix = f"DUSP1_F_Final_MG3_Abs4"

# full run, no per-spot files, but summary files with prefix
crops, meta = sampler.run(
    save_dir=save_dir,
    display=0,
    save_individual=False,
    save_summary=True,
    file_prefix=prefix,
    pad=pad,
    cells_per_quad=1,
    spots_per_cell=20,
    spotChannel=0
)

    # DUSP1 Replica M 3hr 100nM time-sweep Partial

In [ ]:
am = DUSP1AnalysisManager(location=loc, log_location=log_location, mac=True) 
am.select_analysis('DUSP1_M_Final')

spots_df = am.select_datasets("spotresults", dtype="dataframe")
clusters_df = am.select_datasets("clusterresults", dtype="dataframe")
props_df = am.select_datasets("cell_properties", dtype="dataframe")

snr_df = SNRAnalysis(spots_df, props_df, clusters_df, abs_threshold=abs_threshold, mg_threshold=mg_threshold)

merged_spots_df, merged_clusters_df, merged_cellprops_df = snr_df.get_results()

# Create an instance of the DUSP1Measurement class.
dusp = DUSP1Measurement(merged_spots_df, merged_clusters_df, merged_cellprops_df)

# Process the data with a chosen threshold method
cell_level_results = dusp.measure(abs_threshold=abs_threshold, mg_threshold=mg_threshold)

# Add replica level unique IDs for 'unique_cell_id', 'unique_spot_id', and 'unique_cluster_id'
# Get number of digits in the max unique_cell_id
max_id = merged_cellprops_df['unique_cell_id'].max()
num_digits = len(str(max_id))

# Calculate multiplier to add a '1' followed by the right number of zeroes - prefix is specific for each experiment (e.g., repD:1, repE:2, etc.)
rep_prefix = 40
prefix = rep_prefix ** num_digits  # e.g., if max_id = 30245 → prefix = 100000

# Apply prefix to all related DataFrames
merged_spots_df['unique_cell_id'] += prefix
merged_clusters_df['unique_cell_id'] += prefix
merged_cellprops_df['unique_cell_id'] += prefix
cell_level_results['unique_cell_id'] += prefix

# Repeat for unique_spot_id and unique_cluster_id
max_spot_id = merged_spots_df['unique_spot_id'].max()
spot_prefix = rep_prefix ** len(str(max_spot_id))
merged_spots_df['unique_spot_id'] += spot_prefix

max_cluster_id = merged_clusters_df['unique_cluster_id'].max()
cluster_prefix = rep_prefix ** len(str(max_cluster_id))
merged_clusters_df['unique_cluster_id'] += cluster_prefix

# Save all results to CSV
rep_string = 'DUSP1_M_Final'
output_dir = df_outut
os.makedirs(output_dir, exist_ok=True)
cell_level_results.to_csv(os.path.join(output_dir, f"{rep_string}_cell_level_results_MG{mg_threshold}_Abs{abs_threshold}_{date_str}.csv"), index=False)
merged_spots_df.to_csv(os.path.join(output_dir, f"{rep_string}_merged_spots_df_MG{mg_threshold}_Abs{abs_threshold}_{date_str}.csv"), index=False)
merged_clusters_df.to_csv(os.path.join(output_dir, f"{rep_string}_merged_clusters_df_MG{mg_threshold}_Abs{abs_threshold}_{date_str}.csv"), index=False)
merged_cellprops_df.to_csv(os.path.join(output_dir, f"{rep_string}_merged_cellprops_df_MG{mg_threshold}_Abs{abs_threshold}_{date_str}.csv"), index=False)

# instantiate sampler
sampler = SpotCropSampler(
    spots_df=merged_spots_df,
    clusters_df=merged_clusters_df,
    cellprops_df=merged_cellprops_df,
    mount_prefix="/Volumes/share"
)

# # — TEST RUN: only 3 displays, no files written —
# crops, meta = sampler.run(
#     save_dir=save_dir,
#     display=3,
#     save_individual=False,
#     save_summary=False,
#     pad=pad,
#     cells_per_quad=1,
#     spots_per_cell=20,
#     spotChannel=0
# )

# choose a prefix for this experiment
prefix = f"DUSP1_M_Final_MG3_Abs4"

# full run, no per-spot files, but summary files with prefix
crops, meta = sampler.run(
    save_dir=save_dir,
    display=0,
    save_individual=False,
    save_summary=True,
    file_prefix=prefix,
    pad=pad,
    cells_per_quad=1,
    spots_per_cell=20,
    spotChannel=0
)

    # DUSP1 Replica N 3hr 100nM time-sweep Partial

In [ ]:
am = DUSP1AnalysisManager(location=loc, log_location=log_location, mac=True) 
am.select_analysis('DUSP1_N_Final2')

spots_df = am.select_datasets("spotresults", dtype="dataframe")
clusters_df = am.select_datasets("clusterresults", dtype="dataframe")
props_df = am.select_datasets("cell_properties", dtype="dataframe")

snr_df = SNRAnalysis(spots_df, props_df, clusters_df, abs_threshold=abs_threshold, mg_threshold=mg_threshold)

merged_spots_df, merged_clusters_df, merged_cellprops_df = snr_df.get_results()

# Create an instance of the DUSP1Measurement class.
dusp = DUSP1Measurement(merged_spots_df, merged_clusters_df, merged_cellprops_df)

# Process the data with a chosen threshold method
cell_level_results = dusp.measure(abs_threshold=abs_threshold, mg_threshold=mg_threshold)

# Add replica level unique IDs for 'unique_cell_id', 'unique_spot_id', and 'unique_cluster_id'
# Get number of digits in the max unique_cell_id
max_id = merged_cellprops_df['unique_cell_id'].max()
num_digits = len(str(max_id))

# Calculate multiplier to add a '1' followed by the right number of zeroes - prefix is specific for each experiment (e.g., repD:1, repE:2, etc.)
rep_prefix = 50
prefix = rep_prefix ** num_digits  # e.g., if max_id = 30245 → prefix = 100000

# Apply prefix to all related DataFrames
merged_spots_df['unique_cell_id'] += prefix
merged_clusters_df['unique_cell_id'] += prefix
merged_cellprops_df['unique_cell_id'] += prefix
cell_level_results['unique_cell_id'] += prefix

# Repeat for unique_spot_id and unique_cluster_id
max_spot_id = merged_spots_df['unique_spot_id'].max()
spot_prefix = rep_prefix ** len(str(max_spot_id))
merged_spots_df['unique_spot_id'] += spot_prefix

max_cluster_id = merged_clusters_df['unique_cluster_id'].max()
cluster_prefix = rep_prefix ** len(str(max_cluster_id))
merged_clusters_df['unique_cluster_id'] += cluster_prefix

# Save all results to CSV
rep_string = 'DUSP1_N_Final'
output_dir = df_outut
os.makedirs(output_dir, exist_ok=True)
cell_level_results.to_csv(os.path.join(output_dir, f"{rep_string}_cell_level_results_MG{mg_threshold}_Abs{abs_threshold}_{date_str}.csv"), index=False)
merged_spots_df.to_csv(os.path.join(output_dir, f"{rep_string}_merged_spots_df_MG{mg_threshold}_Abs{abs_threshold}_{date_str}.csv"), index=False)
merged_clusters_df.to_csv(os.path.join(output_dir, f"{rep_string}_merged_clusters_df_MG{mg_threshold}_Abs{abs_threshold}_{date_str}.csv"), index=False)
merged_cellprops_df.to_csv(os.path.join(output_dir, f"{rep_string}_merged_cellprops_df_MG{mg_threshold}_Abs{abs_threshold}_{date_str}.csv"), index=False)

# instantiate sampler
sampler = SpotCropSampler(
    spots_df=merged_spots_df,
    clusters_df=merged_clusters_df,
    cellprops_df=merged_cellprops_df,
    mount_prefix="/Volumes/share"
)

# # — TEST RUN: only 3 displays, no files written —
# crops, meta = sampler.run(
#     save_dir=save_dir,
#     display=3,
#     save_individual=False,
#     save_summary=False,
#     pad=pad,
#     cells_per_quad=1,
#     spots_per_cell=20,
#     spotChannel=0
# )

# choose a prefix for this experiment
prefix = f"DUSP1_N_Final_MG3_Abs4"

# full run, no per-spot files, but summary files with prefix
crops, meta = sampler.run(
    save_dir=save_dir,
    display=0,
    save_individual=False,
    save_summary=True,
    file_prefix=prefix,
    pad=pad,
    cells_per_quad=1,
    spots_per_cell=20,
    spotChannel=0
)

    DUSP1 75min Concentration-sweep Replica 1

In [ ]:
am = DUSP1AnalysisManager(location=loc, log_location=log_location, mac=True) 
am.select_analysis('DUSP1_CS_R1_Final3')

spots_df = am.select_datasets("spotresults", dtype="dataframe")
clusters_df = am.select_datasets("clusterresults", dtype="dataframe")
props_df = am.select_datasets("cell_properties", dtype="dataframe")

snr_df = SNRAnalysis(spots_df, props_df, clusters_df, abs_threshold=abs_threshold, mg_threshold=mg_threshold)

merged_spots_df, merged_clusters_df, merged_cellprops_df = snr_df.get_results()

# Create an instance of the DUSP1Measurement class.
dusp = DUSP1Measurement(merged_spots_df, merged_clusters_df, merged_cellprops_df)

# Process the data with a chosen threshold method
cell_level_results = dusp.measure(abs_threshold=abs_threshold, mg_threshold=mg_threshold)

# Add replica level unique IDs for 'unique_cell_id', 'unique_spot_id', and 'unique_cluster_id'
# Get number of digits in the max unique_cell_id
max_id = merged_cellprops_df['unique_cell_id'].max()
num_digits = len(str(max_id))

# Calculate multiplier to add a '1' followed by the right number of zeroes - prefix is specific for each experiment (e.g., repD:1, repE:2, etc.)
rep_prefix = 60
prefix = rep_prefix ** num_digits  # e.g., if max_id = 30245 → prefix = 100000

# Apply prefix to all related DataFrames
merged_spots_df['unique_cell_id'] += prefix
merged_clusters_df['unique_cell_id'] += prefix
merged_cellprops_df['unique_cell_id'] += prefix
cell_level_results['unique_cell_id'] += prefix

# Repeat for unique_spot_id and unique_cluster_id
max_spot_id = merged_spots_df['unique_spot_id'].max()
spot_prefix = rep_prefix ** len(str(max_spot_id))
merged_spots_df['unique_spot_id'] += spot_prefix

max_cluster_id = merged_clusters_df['unique_cluster_id'].max()
cluster_prefix = rep_prefix ** len(str(max_cluster_id))
merged_clusters_df['unique_cluster_id'] += cluster_prefix

# Save all results to CSV
rep_string = 'DUSP1_G_Final'
output_dir = df_outut
os.makedirs(output_dir, exist_ok=True)
cell_level_results.to_csv(os.path.join(output_dir, f"{rep_string}_cell_level_results_MG{mg_threshold}_Abs{abs_threshold}_{date_str}.csv"), index=False)
merged_spots_df.to_csv(os.path.join(output_dir, f"{rep_string}_merged_spots_df_MG{mg_threshold}_Abs{abs_threshold}_{date_str}.csv"), index=False)
merged_clusters_df.to_csv(os.path.join(output_dir, f"{rep_string}_merged_clusters_df_MG{mg_threshold}_Abs{abs_threshold}_{date_str}.csv"), index=False)
merged_cellprops_df.to_csv(os.path.join(output_dir, f"{rep_string}_merged_cellprops_df_MG{mg_threshold}_Abs{abs_threshold}_{date_str}.csv"), index=False)

# instantiate sampler
sampler = SpotCropSampler(
    spots_df=merged_spots_df,
    clusters_df=merged_clusters_df,
    cellprops_df=merged_cellprops_df,
    mount_prefix="/Volumes/share"
)

# # — TEST RUN: only 3 displays, no files written —
# crops, meta = sampler.run(
#     save_dir=save_dir,
#     display=3,
#     save_individual=False,
#     save_summary=False,
#     pad=pad,
#     cells_per_quad=1,
#     spots_per_cell=20,
#     spotChannel=0
# )

# choose a prefix for this experiment
prefix = f"DUSP1_G_Final_MG3_Abs4"

# full run, no per-spot files, but summary files with prefix
crops, meta = sampler.run(
    save_dir=save_dir,
    display=0,
    save_individual=False,
    save_summary=True,
    file_prefix=prefix,
    pad=pad,
    cells_per_quad=1,
    spots_per_cell=20,
    spotChannel=0
)

    DUSP1 75min Concentration-sweep Replica 2

In [ ]:
am = DUSP1AnalysisManager(location=loc, log_location=log_location, mac=True) 
am.select_analysis('DUSP1_H_Final')

spots_df = am.select_datasets("spotresults", dtype="dataframe")
clusters_df = am.select_datasets("clusterresults", dtype="dataframe")
props_df = am.select_datasets("cell_properties", dtype="dataframe")

snr_df = SNRAnalysis(spots_df, props_df, clusters_df, abs_threshold=abs_threshold, mg_threshold=mg_threshold)

merged_spots_df, merged_clusters_df, merged_cellprops_df = snr_df.get_results()

# Create an instance of the DUSP1Measurement class.
dusp = DUSP1Measurement(merged_spots_df, merged_clusters_df, merged_cellprops_df)

# Process the data with a chosen threshold method
cell_level_results = dusp.measure(abs_threshold=abs_threshold, mg_threshold=mg_threshold)

# Add replica level unique IDs for 'unique_cell_id', 'unique_spot_id', and 'unique_cluster_id'
# Get number of digits in the max unique_cell_id
max_id = merged_cellprops_df['unique_cell_id'].max()
num_digits = len(str(max_id))

# Calculate multiplier to add a '1' followed by the right number of zeroes - prefix is specific for each experiment (e.g., repD:1, repE:2, etc.)
rep_prefix = 70
prefix = rep_prefix ** num_digits  # e.g., if max_id = 30245 → prefix = 100000

# Apply prefix to all related DataFrames
merged_spots_df['unique_cell_id'] += prefix
merged_clusters_df['unique_cell_id'] += prefix
merged_cellprops_df['unique_cell_id'] += prefix
cell_level_results['unique_cell_id'] += prefix

# Repeat for unique_spot_id and unique_cluster_id
max_spot_id = merged_spots_df['unique_spot_id'].max()
spot_prefix = rep_prefix ** len(str(max_spot_id))
merged_spots_df['unique_spot_id'] += spot_prefix

max_cluster_id = merged_clusters_df['unique_cluster_id'].max()
cluster_prefix = rep_prefix ** len(str(max_cluster_id))
merged_clusters_df['unique_cluster_id'] += cluster_prefix

# Save all results to CSV
rep_string = 'DUSP1_H_Final'
output_dir = df_outut
os.makedirs(output_dir, exist_ok=True)
cell_level_results.to_csv(os.path.join(output_dir, f"{rep_string}_cell_level_results_MG{mg_threshold}_Abs{abs_threshold}_{date_str}.csv"), index=False)
merged_spots_df.to_csv(os.path.join(output_dir, f"{rep_string}_merged_spots_df_MG{mg_threshold}_Abs{abs_threshold}_{date_str}.csv"), index=False)
merged_clusters_df.to_csv(os.path.join(output_dir, f"{rep_string}_merged_clusters_df_MG{mg_threshold}_Abs{abs_threshold}_{date_str}.csv"), index=False)
merged_cellprops_df.to_csv(os.path.join(output_dir, f"{rep_string}_merged_cellprops_df_MG{mg_threshold}_Abs{abs_threshold}_{date_str}.csv"), index=False)

# instantiate sampler
sampler = SpotCropSampler(
    spots_df=merged_spots_df,
    clusters_df=merged_clusters_df,
    cellprops_df=merged_cellprops_df,
    mount_prefix="/Volumes/share"
)

# # — TEST RUN: only 3 displays, no files written —
# crops, meta = sampler.run(
#     save_dir=save_dir,
#     display=3,
#     save_individual=False,
#     save_summary=False,
#     pad=pad,
#     cells_per_quad=1,
#     spots_per_cell=20,
#     spotChannel=0
# )

# choose a prefix for this experiment
prefix = f"DUSP1_H_Final_MG3_Abs4"

# full run, no per-spot files, but summary files with prefix
crops, meta = sampler.run(
    save_dir=save_dir,
    display=0,
    save_individual=False,
    save_summary=True,
    file_prefix=prefix,
    pad=pad,
    cells_per_quad=1,
    spots_per_cell=20,
    spotChannel=0
)

    DUSP1 75min Concentration-sweep Replica 3

In [ ]:
am = DUSP1AnalysisManager(location=loc, log_location=log_location, mac=True) 
am.select_analysis('DUSP1_I_Final')

spots_df = am.select_datasets("spotresults", dtype="dataframe")
clusters_df = am.select_datasets("clusterresults", dtype="dataframe")
props_df = am.select_datasets("cell_properties", dtype="dataframe")

snr_df = SNRAnalysis(spots_df, props_df, clusters_df, abs_threshold=abs_threshold, mg_threshold=mg_threshold)

merged_spots_df, merged_clusters_df, merged_cellprops_df = snr_df.get_results()

# Create an instance of the DUSP1Measurement class.
dusp = DUSP1Measurement(merged_spots_df, merged_clusters_df, merged_cellprops_df)

# Process the data with a chosen threshold method
cell_level_results = dusp.measure(abs_threshold=abs_threshold, mg_threshold=mg_threshold)

# Add replica level unique IDs for 'unique_cell_id', 'unique_spot_id', and 'unique_cluster_id'
# Get number of digits in the max unique_cell_id
max_id = merged_cellprops_df['unique_cell_id'].max()
num_digits = len(str(max_id))

# Calculate multiplier to add a '1' followed by the right number of zeroes - prefix is specific for each experiment (e.g., repD:1, repE:2, etc.)
rep_prefix = 80
prefix = rep_prefix ** num_digits  # e.g., if max_id = 30245 → prefix = 100000

# Apply prefix to all related DataFrames
merged_spots_df['unique_cell_id'] += prefix
merged_clusters_df['unique_cell_id'] += prefix
merged_cellprops_df['unique_cell_id'] += prefix
cell_level_results['unique_cell_id'] += prefix

# Repeat for unique_spot_id and unique_cluster_id
max_spot_id = merged_spots_df['unique_spot_id'].max()
spot_prefix = rep_prefix ** len(str(max_spot_id))
merged_spots_df['unique_spot_id'] += spot_prefix

max_cluster_id = merged_clusters_df['unique_cluster_id'].max()
cluster_prefix = rep_prefix ** len(str(max_cluster_id))
merged_clusters_df['unique_cluster_id'] += cluster_prefix

# Save all results to CSV
rep_string = 'DUSP1_I_Final'
output_dir = df_outut
os.makedirs(output_dir, exist_ok=True)
cell_level_results.to_csv(os.path.join(output_dir, f"{rep_string}_cell_level_results_MG{mg_threshold}_Abs{abs_threshold}_{date_str}.csv"), index=False)
merged_spots_df.to_csv(os.path.join(output_dir, f"{rep_string}_merged_spots_df_MG{mg_threshold}_Abs{abs_threshold}_{date_str}.csv"), index=False)
merged_clusters_df.to_csv(os.path.join(output_dir, f"{rep_string}_merged_clusters_df_MG{mg_threshold}_Abs{abs_threshold}_{date_str}.csv"), index=False)
merged_cellprops_df.to_csv(os.path.join(output_dir, f"{rep_string}_merged_cellprops_df_MG{mg_threshold}_Abs{abs_threshold}_{date_str}.csv"), index=False)

# instantiate sampler
sampler = SpotCropSampler(
    spots_df=merged_spots_df,
    clusters_df=merged_clusters_df,
    cellprops_df=merged_cellprops_df,
    mount_prefix="/Volumes/share"
)

# # — TEST RUN: only 3 displays, no files written —
# crops, meta = sampler.run(
#     save_dir=save_dir,
#     display=3,
#     save_individual=False,
#     save_summary=False,
#     pad=pad,
#     cells_per_quad=1,
#     spots_per_cell=20,
#     spotChannel=0
# )

# choose a prefix for this experiment
prefix = f"DUSP1_I_Final_MG3_Abs4"

# full run, no per-spot files, but summary files with prefix
crops, meta = sampler.run(
    save_dir=save_dir,
    display=0,
    save_individual=False,
    save_summary=True,
    file_prefix=prefix,
    pad=pad,
    cells_per_quad=1,
    spots_per_cell=20,
    spotChannel=0
)

    DUSP1 0.3, 1, 10nM Dex 3hr Time-sweep Replica 1

In [ ]:
am = DUSP1AnalysisManager(location=loc, log_location=log_location, mac=True) 
am.select_analysis('DUSP1_TCS_R1_Final3')

spots_df = am.select_datasets("spotresults", dtype="dataframe")
clusters_df = am.select_datasets("clusterresults", dtype="dataframe")
props_df = am.select_datasets("cell_properties", dtype="dataframe")

snr_df = SNRAnalysis(spots_df, props_df, clusters_df, abs_threshold=abs_threshold, mg_threshold=mg_threshold)

merged_spots_df, merged_clusters_df, merged_cellprops_df = snr_df.get_results()

# Create an instance of the DUSP1Measurement class.
dusp = DUSP1Measurement(merged_spots_df, merged_clusters_df, merged_cellprops_df)

# Process the data with a chosen threshold method
cell_level_results = dusp.measure(abs_threshold=abs_threshold, mg_threshold=mg_threshold)

# Add replica level unique IDs for 'unique_cell_id', 'unique_spot_id', and 'unique_cluster_id'
# Get number of digits in the max unique_cell_id
max_id = merged_cellprops_df['unique_cell_id'].max()
num_digits = len(str(max_id))

# Calculate multiplier to add a '1' followed by the right number of zeroes - prefix is specific for each experiment (e.g., repD:1, repE:2, etc.)
rep_prefix = 90
prefix = rep_prefix ** num_digits  # e.g., if max_id = 30245 → prefix = 100000

# Apply prefix to all related DataFrames
merged_spots_df['unique_cell_id'] += prefix
merged_clusters_df['unique_cell_id'] += prefix
merged_cellprops_df['unique_cell_id'] += prefix
cell_level_results['unique_cell_id'] += prefix

# Repeat for unique_spot_id and unique_cluster_id
max_spot_id = merged_spots_df['unique_spot_id'].max()
spot_prefix = rep_prefix ** len(str(max_spot_id))
merged_spots_df['unique_spot_id'] += spot_prefix

max_cluster_id = merged_clusters_df['unique_cluster_id'].max()
cluster_prefix = rep_prefix ** len(str(max_cluster_id))
merged_clusters_df['unique_cluster_id'] += cluster_prefix

# Save all results to CSV
rep_string = 'DUSP1_J_Final'
output_dir = df_outut
os.makedirs(output_dir, exist_ok=True)
cell_level_results.to_csv(os.path.join(output_dir, f"{rep_string}_cell_level_results_MG{mg_threshold}_Abs{abs_threshold}_{date_str}.csv"), index=False)
merged_spots_df.to_csv(os.path.join(output_dir, f"{rep_string}_merged_spots_df_MG{mg_threshold}_Abs{abs_threshold}_{date_str}.csv"), index=False)
merged_clusters_df.to_csv(os.path.join(output_dir, f"{rep_string}_merged_clusters_df_MG{mg_threshold}_Abs{abs_threshold}_{date_str}.csv"), index=False)
merged_cellprops_df.to_csv(os.path.join(output_dir, f"{rep_string}_merged_cellprops_df_MG{mg_threshold}_Abs{abs_threshold}_{date_str}.csv"), index=False)

# instantiate sampler
sampler = SpotCropSampler(
    spots_df=merged_spots_df,
    clusters_df=merged_clusters_df,
    cellprops_df=merged_cellprops_df,
    mount_prefix="/Volumes/share"
)

# # — TEST RUN: only 3 displays, no files written —
# crops, meta = sampler.run(
#     save_dir=save_dir,
#     display=3,
#     save_individual=False,
#     save_summary=False,
#     pad=pad,
#     cells_per_quad=1,
#     spots_per_cell=20,
#     spotChannel=0
# )

# choose a prefix for this experiment
prefix = f"DUSP1_J_Final_MG3_Abs4"

# full run, no per-spot files, but summary files with prefix
crops, meta = sampler.run(
    save_dir=save_dir,
    display=0,
    save_individual=False,
    save_summary=True,
    file_prefix=prefix,
    pad=pad,
    cells_per_quad=1,
    spots_per_cell=20,
    spotChannel=0
)

    DUSP1 0.3, 1, 10nM Dex 3hr Time-sweep Replica 2


In [ ]:
am = DUSP1AnalysisManager(location=loc, log_location=log_location, mac=True) 
am.select_analysis('DUSP1_K_Final')

spots_df = am.select_datasets("spotresults", dtype="dataframe")
clusters_df = am.select_datasets("clusterresults", dtype="dataframe")
props_df = am.select_datasets("cell_properties", dtype="dataframe")

snr_df = SNRAnalysis(spots_df, props_df, clusters_df, abs_threshold=abs_threshold, mg_threshold=mg_threshold)

merged_spots_df, merged_clusters_df, merged_cellprops_df = snr_df.get_results()

# Create an instance of the DUSP1Measurement class.
dusp = DUSP1Measurement(merged_spots_df, merged_clusters_df, merged_cellprops_df)

# Process the data with a chosen threshold method
cell_level_results = dusp.measure(abs_threshold=abs_threshold, mg_threshold=mg_threshold)

# Add replica level unique IDs for 'unique_cell_id', 'unique_spot_id', and 'unique_cluster_id'
# Get number of digits in the max unique_cell_id
max_id = merged_cellprops_df['unique_cell_id'].max()
num_digits = len(str(max_id))

# Calculate multiplier to add a '1' followed by the right number of zeroes - prefix is specific for each experiment (e.g., repD:1, repE:2, etc.)
rep_prefix = 11
prefix = rep_prefix * (10** num_digits)  # e.g., if max_id = 30245 → prefix = 1100000

# Apply prefix to all related DataFrames
merged_spots_df['unique_cell_id'] += prefix
merged_clusters_df['unique_cell_id'] += prefix
merged_cellprops_df['unique_cell_id'] += prefix
cell_level_results['unique_cell_id'] += prefix

# Repeat for unique_spot_id and unique_cluster_id
max_spot_id = merged_spots_df['unique_spot_id'].max()
spot_prefix = rep_prefix ** len(str(max_spot_id))
merged_spots_df['unique_spot_id'] += spot_prefix

max_cluster_id = merged_clusters_df['unique_cluster_id'].max()
cluster_prefix = rep_prefix ** len(str(max_cluster_id))
merged_clusters_df['unique_cluster_id'] += cluster_prefix

# Save all results to CSV
rep_string = 'DUSP1_k_Final'
output_dir = df_outut
os.makedirs(output_dir, exist_ok=True)
cell_level_results.to_csv(os.path.join(output_dir, f"{rep_string}_cell_level_results_MG{mg_threshold}_Abs{abs_threshold}_{date_str}.csv"), index=False)
merged_spots_df.to_csv(os.path.join(output_dir, f"{rep_string}_merged_spots_df_MG{mg_threshold}_Abs{abs_threshold}_{date_str}.csv"), index=False)
merged_clusters_df.to_csv(os.path.join(output_dir, f"{rep_string}_merged_clusters_df_MG{mg_threshold}_Abs{abs_threshold}_{date_str}.csv"), index=False)
merged_cellprops_df.to_csv(os.path.join(output_dir, f"{rep_string}_merged_cellprops_df_MG{mg_threshold}_Abs{abs_threshold}_{date_str}.csv"), index=False)

# instantiate sampler
sampler = SpotCropSampler(
    spots_df=merged_spots_df,
    clusters_df=merged_clusters_df,
    cellprops_df=merged_cellprops_df,
    mount_prefix="/Volumes/share"
)

# # — TEST RUN: only 3 displays, no files written —
# crops, meta = sampler.run(
#     save_dir=save_dir,
#     display=3,
#     save_individual=False,
#     save_summary=False,
#     pad=pad,
#     cells_per_quad=1,
#     spots_per_cell=20,
#     spotChannel=0
# )

# choose a prefix for this experiment
prefix = f"DUSP1_K_Final_MG3_Abs4"

# full run, no per-spot files, but summary files with prefix
crops, meta = sampler.run(
    save_dir=save_dir,
    display=0,
    save_individual=False,
    save_summary=True,
    file_prefix=prefix,
    pad=pad,
    cells_per_quad=1,
    spots_per_cell=20,
    spotChannel=0
)

    DUSP1 0.3, 1, 10nM Dex 3hr Time-sweep Replica 3

In [ ]:
am = DUSP1AnalysisManager(location=loc, log_location=log_location, mac=True) 
am.select_analysis('DUSP1_L_Final')

spots_df = am.select_datasets("spotresults", dtype="dataframe")
clusters_df = am.select_datasets("clusterresults", dtype="dataframe")
props_df = am.select_datasets("cell_properties", dtype="dataframe")

snr_df = SNRAnalysis(spots_df, props_df, clusters_df, abs_threshold=abs_threshold, mg_threshold=mg_threshold)

merged_spots_df, merged_clusters_df, merged_cellprops_df = snr_df.get_results()

# Create an instance of the DUSP1Measurement class.
dusp = DUSP1Measurement(merged_spots_df, merged_clusters_df, merged_cellprops_df)

# Process the data with a chosen threshold method
cell_level_results = dusp.measure(abs_threshold=abs_threshold, mg_threshold=mg_threshold)

# Add replica level unique IDs for 'unique_cell_id', 'unique_spot_id', and 'unique_cluster_id'
# Get number of digits in the max unique_cell_id
max_id = merged_cellprops_df['unique_cell_id'].max()
num_digits = len(str(max_id))

# Calculate multiplier to add a '1' followed by the right number of zeroes - prefix is specific for each experiment (e.g., repD:1, repE:2, etc.)
rep_prefix = 12
prefix = rep_prefix * (10** num_digits)  # e.g., if max_id = 30245 → prefix = 100000

# Apply prefix to all related DataFrames
merged_spots_df['unique_cell_id'] += prefix
merged_clusters_df['unique_cell_id'] += prefix
merged_cellprops_df['unique_cell_id'] += prefix
cell_level_results['unique_cell_id'] += prefix

# Repeat for unique_spot_id and unique_cluster_id
max_spot_id = merged_spots_df['unique_spot_id'].max()
spot_prefix = rep_prefix ** len(str(max_spot_id))
merged_spots_df['unique_spot_id'] += spot_prefix

max_cluster_id = merged_clusters_df['unique_cluster_id'].max()
cluster_prefix = rep_prefix ** len(str(max_cluster_id))
merged_clusters_df['unique_cluster_id'] += cluster_prefix

# Save all results to CSV
rep_string = 'DUSP1_L_Final'
output_dir = df_outut
os.makedirs(output_dir, exist_ok=True)
cell_level_results.to_csv(os.path.join(output_dir, f"{rep_string}_cell_level_results_MG{mg_threshold}_Abs{abs_threshold}_{date_str}.csv"), index=False)
merged_spots_df.to_csv(os.path.join(output_dir, f"{rep_string}_merged_spots_df_MG{mg_threshold}_Abs{abs_threshold}_{date_str}.csv"), index=False)
merged_clusters_df.to_csv(os.path.join(output_dir, f"{rep_string}_merged_clusters_df_MG{mg_threshold}_Abs{abs_threshold}_{date_str}.csv"), index=False)
merged_cellprops_df.to_csv(os.path.join(output_dir, f"{rep_string}_merged_cellprops_df_MG{mg_threshold}_Abs{abs_threshold}_{date_str}.csv"), index=False)

# instantiate sampler
sampler = SpotCropSampler(
    spots_df=merged_spots_df,
    clusters_df=merged_clusters_df,
    cellprops_df=merged_cellprops_df,
    mount_prefix="/Volumes/share"
)

# # — TEST RUN: only 3 displays, no files written —
# crops, meta = sampler.run(
#     save_dir=save_dir,
#     display=3,
#     save_individual=False,
#     save_summary=False,
#     pad=pad,
#     cells_per_quad=1,
#     spots_per_cell=20,
#     spotChannel=0
# )

# choose a prefix for this experiment
prefix = f"DUSP1_L_Final_MG3_Abs4"

# full run, no per-spot files, but summary files with prefix
crops, meta = sampler.run(
    save_dir=save_dir,
    display=0,
    save_individual=False,
    save_summary=True,
    file_prefix=prefix,
    pad=pad,
    cells_per_quad=1,
    spots_per_cell=20,
    spotChannel=0
)

    DUSP1 100nM Dex & 5µM TPL Time-sweep Replica 1

In [ ]:
am = DUSP1AnalysisManager(location=loc, log_location=log_location, mac=True) 
am.select_analysis('DUSP1_O_Final')

spots_df = am.select_datasets("spotresults", dtype="dataframe")
clusters_df = am.select_datasets("clusterresults", dtype="dataframe")
props_df = am.select_datasets("cell_properties", dtype="dataframe")

snr_df = SNRAnalysis(spots_df, props_df, clusters_df, abs_threshold=abs_threshold, mg_threshold=mg_threshold)

merged_spots_df, merged_clusters_df, merged_cellprops_df = snr_df.get_results()

# Create an instance of the DUSP1Measurement class.
dusp = DUSP1Measurement(merged_spots_df, merged_clusters_df, merged_cellprops_df)

# Process the data with a chosen threshold method
cell_level_results = dusp.measure(abs_threshold=abs_threshold, mg_threshold=mg_threshold)

# Add replica level unique IDs for 'unique_cell_id', 'unique_spot_id', and 'unique_cluster_id'
# Get number of digits in the max unique_cell_id
max_id = merged_cellprops_df['unique_cell_id'].max()
num_digits = len(str(max_id))

# Calculate multiplier to add a '1' followed by the right number of zeroes - prefix is specific for each experiment (e.g., repD:1, repE:2, etc.)
rep_prefix = 13
prefix = rep_prefix * (10** num_digits)  # e.g., if max_id = 30245 → prefix = 100000

# Apply prefix to all related DataFrames
merged_spots_df['unique_cell_id'] += prefix
merged_clusters_df['unique_cell_id'] += prefix
merged_cellprops_df['unique_cell_id'] += prefix
cell_level_results['unique_cell_id'] += prefix

# Repeat for unique_spot_id and unique_cluster_id
max_spot_id = merged_spots_df['unique_spot_id'].max()
spot_prefix = rep_prefix ** len(str(max_spot_id))
merged_spots_df['unique_spot_id'] += spot_prefix

max_cluster_id = merged_clusters_df['unique_cluster_id'].max()
cluster_prefix = rep_prefix ** len(str(max_cluster_id))
merged_clusters_df['unique_cluster_id'] += cluster_prefix

# Save all results to CSV
rep_string = 'DUSP1_O_Final'
output_dir = df_outut
os.makedirs(output_dir, exist_ok=True)
cell_level_results.to_csv(os.path.join(output_dir, f"{rep_string}_cell_level_results_MG{mg_threshold}_Abs{abs_threshold}_{date_str}.csv"), index=False)
merged_spots_df.to_csv(os.path.join(output_dir, f"{rep_string}_merged_spots_df_MG{mg_threshold}_Abs{abs_threshold}_{date_str}.csv"), index=False)
merged_clusters_df.to_csv(os.path.join(output_dir, f"{rep_string}_merged_clusters_df_MG{mg_threshold}_Abs{abs_threshold}_{date_str}.csv"), index=False)
merged_cellprops_df.to_csv(os.path.join(output_dir, f"{rep_string}_merged_cellprops_df_MG{mg_threshold}_Abs{abs_threshold}_{date_str}.csv"), index=False)

# instantiate sampler
sampler = SpotCropSampler(
    spots_df=merged_spots_df,
    clusters_df=merged_clusters_df,
    cellprops_df=merged_cellprops_df,
    mount_prefix="/Volumes/share"
)

# # — TEST RUN: only 3 displays, no files written —
# crops, meta = sampler.run(
#     save_dir=save_dir,
#     display=3,
#     save_individual=False,
#     save_summary=False,
#     pad=pad,
#     cells_per_quad=1,
#     spots_per_cell=20,
#     spotChannel=0
# )

# choose a prefix for this experiment
prefix = f"DUSP1_O_Final_MG3_Abs4"

# full run, no per-spot files, but summary files with prefix
crops, meta = sampler.run(
    save_dir=save_dir,
    display=0,
    save_individual=False,
    save_summary=True,
    file_prefix=prefix,
    pad=pad,
    cells_per_quad=1,
    spots_per_cell=20,
    spotChannel=0
)

    DUSP1 100nM Dex & 5µM TPL Time-sweep Replica 2

In [ ]:
am = DUSP1AnalysisManager(location=loc, log_location=log_location, mac=True) 
am.select_analysis('DUSP1_P_Final2')

spots_df = am.select_datasets("spotresults", dtype="dataframe")
clusters_df = am.select_datasets("clusterresults", dtype="dataframe")
props_df = am.select_datasets("cell_properties", dtype="dataframe")

snr_df = SNRAnalysis(spots_df, props_df, clusters_df, abs_threshold=abs_threshold, mg_threshold=mg_threshold)

merged_spots_df, merged_clusters_df, merged_cellprops_df = snr_df.get_results()

# Create an instance of the DUSP1Measurement class.
dusp = DUSP1Measurement(merged_spots_df, merged_clusters_df, merged_cellprops_df)

# Process the data with a chosen threshold method
cell_level_results = dusp.measure(abs_threshold=abs_threshold, mg_threshold=mg_threshold)

# Add replica level unique IDs for 'unique_cell_id', 'unique_spot_id', and 'unique_cluster_id'
# Get number of digits in the max unique_cell_id
max_id = merged_cellprops_df['unique_cell_id'].max()
num_digits = len(str(max_id))

# Calculate multiplier to add a '1' followed by the right number of zeroes - prefix is specific for each experiment (e.g., repD:1, repE:2, etc.)
rep_prefix = 14
prefix = rep_prefix * (10** num_digits)  # e.g., if max_id = 30245 → prefix = 100000

# Apply prefix to all related DataFrames
merged_spots_df['unique_cell_id'] += prefix
merged_clusters_df['unique_cell_id'] += prefix
merged_cellprops_df['unique_cell_id'] += prefix
cell_level_results['unique_cell_id'] += prefix

# Repeat for unique_spot_id and unique_cluster_id
max_spot_id = merged_spots_df['unique_spot_id'].max()
spot_prefix = rep_prefix ** len(str(max_spot_id))
merged_spots_df['unique_spot_id'] += spot_prefix

max_cluster_id = merged_clusters_df['unique_cluster_id'].max()
cluster_prefix = rep_prefix ** len(str(max_cluster_id))
merged_clusters_df['unique_cluster_id'] += cluster_prefix

# Save all results to CSV
rep_string = 'DUSP1_P_Final'
output_dir = df_outut
os.makedirs(output_dir, exist_ok=True)
cell_level_results.to_csv(os.path.join(output_dir, f"{rep_string}_cell_level_results_MG{mg_threshold}_Abs{abs_threshold}_{date_str}.csv"), index=False)
merged_spots_df.to_csv(os.path.join(output_dir, f"{rep_string}_merged_spots_df_MG{mg_threshold}_Abs{abs_threshold}_{date_str}.csv"), index=False)
merged_clusters_df.to_csv(os.path.join(output_dir, f"{rep_string}_merged_clusters_df_MG{mg_threshold}_Abs{abs_threshold}_{date_str}.csv"), index=False)
merged_cellprops_df.to_csv(os.path.join(output_dir, f"{rep_string}_merged_cellprops_df_MG{mg_threshold}_Abs{abs_threshold}_{date_str}.csv"), index=False)

# instantiate sampler
sampler = SpotCropSampler(
    spots_df=merged_spots_df,
    clusters_df=merged_clusters_df,
    cellprops_df=merged_cellprops_df,
    mount_prefix="/Volumes/share"
)

# # — TEST RUN: only 3 displays, no files written —
# crops, meta = sampler.run(
#     save_dir=save_dir,
#     display=3,
#     save_individual=False,
#     save_summary=False,
#     pad=pad,
#     cells_per_quad=1,
#     spots_per_cell=20,
#     spotChannel=0
# )

# choose a prefix for this experiment
prefix = f"DUSP1_P_Final_MG3_Abs4"

# full run, no per-spot files, but summary files with prefix
crops, meta = sampler.run(
    save_dir=save_dir,
    display=0,
    save_individual=False,
    save_summary=True,
    file_prefix=prefix,
    pad=pad,
    cells_per_quad=1,
    spots_per_cell=20,
    spotChannel=0
)

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# — 1) Load your data —
spots_df = pd.read_csv(
    "/Users/ericron/Desktop/AngelFISH/Publications/Ron_2024/Classification/"
    "DUSP1_D_Final_merged_spots_df_MG3_Abs4_Apr29.csv"
)
clusters_df = pd.read_csv(
    "/Users/ericron/Desktop/AngelFISH/Publications/Ron_2024/Classification/"
    "DUSP1_D_Final_merged_clusters_df_MG3_Abs4_Apr29.csv"
)
cellprops_df = pd.read_csv(
    "/Users/ericron/Desktop/AngelFISH/Publications/Ron_2024/Classification/"
    "DUSP1_D_Final_merged_cellprops_df_MG3_Abs4_Apr29.csv"
)

In [ ]:
# Get info on spots that do not pass MG_pass and mg_lt_snr =1
spots_df.keys()


In [ ]:
# 1) Get every unique_cell_id as strings
all_ids = cellprops_df['unique_cell_id'].astype(str).unique().tolist()

# 2) Find those that start with "10" and end with "619" or "1269"
matching = [
    cid for cid in all_ids
    if cid.startswith('10') and (cid.endswith('619') or cid.endswith('1269'))
]

print("Found these matching IDs:", matching)

# 3) If 'matching' is empty, broaden the search to any containing "619" or "1269":
if not matching:
    broader = [cid for cid in all_ids if '619' in cid or '1269' in cid]
    print("Broader matches (contain '619' or '1269'):", broader)

## Train Classifier

In [ ]:
# ─── 1) Imports & Paths ─────────────────────────────────────────────────────────
import os
import sys
import pandas as pd
import joblib
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import re

# make sure your Analysis_DUSP1_v2.py sits under a 'src/' folder in this ROOT
ROOT = '/Users/ericron/Desktop/AngelFISH'  
sys.path.append(ROOT)

# adjust these to wherever your files really live:
spots_path     = os.path.join(ROOT, 'Publications', 'Ron_2024', 'Classification',
                              'DUSP1_D_Final_merged_spots_df_MG3_Abs4_Apr29.csv')
clusters_path  = os.path.join(ROOT, 'Publications', 'Ron_2024', 'Classification',
                              'DUSP1_D_Final_merged_clusters_df_MG3_Abs4_Apr29.csv')
cellprops_path = os.path.join(ROOT, 'Publications', 'Ron_2024', 'Classification',
                              'DUSP1_D_Final_merged_cellprops_df_MG3_Abs4_Apr29.csv')
model_path     = '/Volumes/share/Users/Eric/DUSP1_SpotCropsV2/global_random_forest.pkl'

# ─── 2) Load DataFrames ────────────────────────────────────────────────────────
merged_spots_df     = pd.read_csv(spots_path)
merged_clusters_df  = pd.read_csv(clusters_path)
merged_cellprops_df = pd.read_csv(cellprops_path)

# ─── 3) Import your classes and load RF model ─────────────────────────────────
from src.Analysis_DUSP1_v2 import DUSP1Measurement, DUSP1_filtering

if os.path.exists(model_path):
    clf = joblib.load(model_path)
    all_feats = list(clf.feature_names_in_)
    # drop the three spot-crop features
    features = [f for f in all_feats 
                if f not in ('num_spots_in_crop', 'num_TS_in_crop', 'num_foci_in_crop')]
    print(f"Loaded RF model ({len(all_feats)} total features).")
    print(f"Using {len(features)} features (crop counts dropped):\n", features)
else:
    clf = None
    features = []
    print("RF model not found; skipping RF-based steps.")

# ─── 4) Run measurement pipeline ───────────────────────────────────────────────
dusp = DUSP1Measurement(
    spots=merged_spots_df,
    clusters=merged_clusters_df,
    cellprops=merged_cellprops_df,
    model=model_path
)

# set your thresholds here:
ABS_THRESHOLD = 4.0
MG_THRESHOLD  = 3.0

results_df = dusp.measure(
    abs_threshold=ABS_THRESHOLD,
    mg_threshold=MG_THRESHOLD
)

# ─── 5) RF filter spots ────────────────────────────────────────────────────────
rf_filter = DUSP1_filtering(method='all')
filtered_spots, removed_spots = rf_filter.apply_spots(
    spots=dusp.spots,
    results=results_df,
    method='all'
)

# ─── 6) Show exactly which columns fed the RF ─────────────────────────────────
if clf:
    print("Features used by RF:", list(clf.feature_names_in_))

# 7) Prepare for PCA & prediction by auto‐mapping model features to df columns
all_feats = list(clf.feature_names_in_) if clf else []

def normalize(name):
    """Lowercase, turn non‐alphanumerics to '_', strip trailing '_<digits>', collapse underscores."""
    s = re.sub(r'[^0-9a-z]+', '_', name.lower())
    s = re.sub(r'_[0-9]+$', '', s)
    return s.strip('_')

# build map: normalized column name -> actual column name
norm2col = { normalize(c): c for c in merged_spots_df.columns }

mapped = {}
missing = []
for feat in all_feats:
    n = normalize(feat)
    if n in norm2col:
        mapped[feat] = norm2col[n]
    else:
        missing.append(feat)

if missing:
    print("Warning: RF expects features not in DataFrame, filling zeros for:", missing)

# now build X with exactly all_feats (missing → 0)
X = pd.DataFrame(index=merged_spots_df.index)
for feat in all_feats:
    if feat in mapped:
        X[feat] = merged_spots_df[mapped[feat]]
    else:
        X[feat] = 0

# ensure numeric
X = X.select_dtypes(include=['int64','float64'])

# annotate original spots with RF output
if clf and not X.empty:
    merged_spots_df['rf_probability'] = clf.predict_proba(X)[:, 1]
    merged_spots_df['rf_prediction']  = (merged_spots_df['rf_probability'] >= 0.5).astype(int)
else:
    merged_spots_df['rf_prediction'] = 1

X.fillna(0, inplace=True)

# 8) PCA projection
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
pc = pca.fit_transform(X)
merged_spots_df['PC1'], merged_spots_df['PC2'] = pc[:,0], pc[:,1]

# 9) Plot kept vs removed
plt.figure(figsize=(6,6))
mask = merged_spots_df['rf_prediction'] == 1
plt.scatter(merged_spots_df.loc[mask,'PC1'],   merged_spots_df.loc[mask,'PC2'],
            c='green', label='Kept', alpha=0.6, edgecolors='k')
plt.scatter(merged_spots_df.loc[~mask,'PC1'], merged_spots_df.loc[~mask,'PC2'],
            c='red',   label='Removed', alpha=0.6, edgecolors='k')
plt.xlabel('PC1'); plt.ylabel('PC2')
plt.legend(); plt.title('PCA of Spots: Kept vs Removed')
plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# ─── 1) Point this at your merged spots CSV ─────────────────────────────────
spots_path = '/Users/ericron/Desktop/AngelFISH/Publications/Ron_2024/Classification/DUSP1_D_Final_merged_spots_df_MG3_Abs4_Apr29.csv'  

# Load the DataFrame
merged_spots_df = pd.read_csv(spots_path)

# 2) select all numeric columns and fill NaNs
numeric_cols = merged_spots_df.select_dtypes(include=[np.number]).columns
X = merged_spots_df[numeric_cols].fillna(0)
drop = ['x_px','y_px','z_px']
X = merged_spots_df[numeric_cols.drop(drop)].fillna(0)

# 3) standardize
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 4) PCA on the scaled data
pca = PCA(n_components=2)
coords = pca.fit_transform(X_scaled)
merged_spots_df['PC1'] = coords[:,0]
merged_spots_df['PC2'] = coords[:,1]

# 5) optionally look at how much variance PC1/PC2 capture:
print("Explained variance ratio:", pca.explained_variance_ratio_)

# 6) plot
plt.figure(figsize=(7,7))
mask = merged_spots_df['MG_pass']
plt.scatter(merged_spots_df.loc[mask,  'PC1'],
            merged_spots_df.loc[mask,  'PC2'],
            c='green', label='MG_pass', alpha=0.6, edgecolors='k')
plt.scatter(merged_spots_df.loc[~mask, 'PC1'],
            merged_spots_df.loc[~mask, 'PC2'],
            c='red',   label='MG_fail', alpha=0.6, edgecolors='k')
plt.xlabel('PC1'); plt.ylabel('PC2')
plt.title('PCA (standardized features): MG_pass vs MG_fail')
plt.legend(); plt.tight_layout(); plt.show()